In [2]:
import torch
from torch.multiprocessing import Pool, Process, set_start_method

In [3]:
u1 = torch.tensor([2, 3, 3])
v1 = torch.tensor([6, 4, 6])
result1 = torch.stack((u1, v1), dim=1)

u2 = torch.tensor([3, 6, 1])
v2 = torch.tensor([6, 2, 4])
result2 = torch.stack((u2, v2), dim=1)

print(result1.shape, result2.shape)
# Check if all UV values in result1 are in result2
mask = torch.isin(result1, result2).all(dim=1)
print(mask)

torch.Size([3, 2]) torch.Size([3, 2])
tensor([True, True, True])


In [4]:
t = torch.tensor([[1, 0, 3], 
                  [4, 0, 2],
                  [4, 0, 2]])
print(t.shape)
mask = torch.all(t == 0, dim=1, keepdim=True)
print(mask, mask.shape)

torch.Size([3, 3])
tensor([[False],
        [False],
        [False]]) torch.Size([3, 1])


In [4]:
t = torch.tensor([
    [
        [1, 0, 3],
        [4, 5, 2],
        [4, 0, 2],
        [4, 0, 2]

    ],
    [
        [1, 0, 3],
        [4, 5, 2],
        [4, 0, 2],
        [4, 0, 2]
    ],
    [
        [1, 0, 3],
        [4, 5, 2],
        [4, 0, 2],
        [4, 0, 2]

    ],
    [
        [1, 0, 3],
        [4, 5, 2],
        [4, 0, 2],
        [4, 0, 2]
    ]
])
print(t.shape)
mask = torch.any(t == 0, dim=2, keepdim=True)
# print(mask, mask.shape)

torch.Size([4, 4, 3])


In [5]:
uv = [[0,1], [1,2], [2,3], [3,3]]
uv = torch.tensor(uv)

print(uv.shape)

rgb = [[0,1,2], [1,2,3], [2,3,4], [3,4,5]]
rgb = torch.tensor(rgb)

t[uv[:,0], uv[:,1]]

torch.Size([4, 2])


tensor([[4, 5, 2],
        [4, 0, 2],
        [4, 0, 2],
        [4, 0, 2]])